# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-21 00:00:00  to  2020-04-22 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,169,167,-2,-1.183432
1,02,295,279,-16,-5.423729
2,03,56,55,-1,-1.785714
3,04,41,31,-10,-24.390244
4,05,41,41,0,0.000000


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2856,2784,-72,-2.521008
94,94,2214,2098,-116,-5.239386
92,92,1923,1917,-6,-0.312012
93,93,1560,1582,22,1.410256
69,69,1213,1204,-9,-0.741962


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -116  Mean:  -3.6979166666666665  Max:  44


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -24.390243902439025  Mean %:  -1.5271884997288405  Max %:  57.14285714285714


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
0,01,169,167,-2,-1.183432
1,02,295,279,-16,-5.423729
2,03,56,55,-1,-1.785714
3,04,41,31,-10,-24.390244
6,07,104,102,-2,-1.923077
7,08,65,59,-6,-9.230769
10,11,69,67,-2,-2.898551
11,12,65,62,-3,-4.615385
13,14,145,142,-3,-2.068966
14,15,30,28,-2,-6.666667


In [11]:
print(stats.query("diff < 0").shape)

(58, 5)


#### <br> Departments with highest evolution of hospitalizations (total number)

In [12]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
95,95,914,958,44,4.814004
93,93,1560,1582,22,1.410256
71,71,207,227,20,9.661836
83,83,307,325,18,5.863192
89,89,104,119,15,14.423077


#### <br> Departments with highest rate of hospitalizations (%)

In [13]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
29,2B,7,11,4,57.142857
89,89,104,119,15,14.423077
71,71,207,227,20,9.661836
83,83,307,325,18,5.863192
95,95,914,958,44,4.814004


#### <br> Focus on some departments

In [14]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2856,2784,-72,-2.521008


In [15]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,207,227,20,9.661836


In [16]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,190,179,-11,-5.789474
